In [ ]:
# dataframe with kinematics
df = pd.DataFrame(data=dict) 
# results = Table.read(dict, format="csv")
# df = results.to_pandas()
df.columns
ra_deg  = df['RA'].values #right ascension [rad] in ICRS
dec_deg = df['DEC'].values #declination [rad] in ICRS
d_kpc   = 1/df['Plx'].values  #distance from Sun [kpc]

#Velocity of stars:
pm_ra_masyr  = df['PMRA'].values #proper motion in direction of right ascension [mas/yr] in ICRS
pm_dec_masyr = df['PMDEC'].values #proper motion in direction of declination [mas/yr] in ICRS
v_los_kms    = df['RV'].values #line-of-sight velocity [km/s]

In [ ]:
#Galactocentric position of the Sun:
X_GC_sun_kpc = ro    #[kpc]
Z_GC_sun_kpc = 0.025 #[kpc] (e.g. Juric et al. 2008)

#Velocity of the Sun w.r.t. the Local Standard of Rest (e.g. Schoenrich et al. 2009):
U_LSR_kms = 11.1  # [km/s]
V_LSR_kms = 12.24 # [km/s]
W_LSR_kms = 7.25  # [km/s]

#circular velocity of the Galactic potential at the radius of the Sun:
vcirc_kms = vo #[km/s] (e.g. Bovy 2015)

#Galactocentric velocity of the Sun:
vX_GC_sun_kms = -U_LSR_kms           # = -U              [km/s]
vY_GC_sun_kms =  V_LSR_kms+vcirc_kms # = V+v_circ(R_Sun) [km/s]
vZ_GC_sun_kms =  W_LSR_kms           # = W               [km/s]

#galpy scale units:
_REFR0 = ro   #[kpc]  --> galpy length unit
_REFV0 = vo #[km/s] --> galpy velocity unit

In [ ]:
#Transformation to Galactic sky coordinates:
#_____(ra,dec) --> Galactic coordinates (l,b):_____

lb = coords.radec_to_lb(
                ra_deg,dec_deg,
                degree=True,
                epoch=None
                )
l_deg = lb[:,0]
b_deg = lb[:,1]
print("galpy ICRS:\t\t(l,b) =\t\t",l_deg,b_deg," [deg]")

#_____(pm_ra,pm_dec) --> (pm_l,pm_b)_____
pmlpmb = coords.pmrapmdec_to_pmllpmbb(
                    pm_ra_masyr,
                    pm_dec_masyr,
                    ra_deg,dec_deg,
                    degree=True,
                    epoch=None
                    )
pml_masyr = pmlpmb[:,0]
pmb_masyr = pmlpmb[:,1]
xyz = coords.lbd_to_XYZ(
                l_deg,b_deg,
                d_kpc,
                degree=True)
X_HC_kpc = xyz[:,0]
Y_HC_kpc = xyz[:,1]
Z_HC_kpc = xyz[:,2]

#_____(v_los,pm_l,pm_b) & (l,b,d) --> (vx,vy,vz)______
vxvyvz = coords.vrpmllpmbb_to_vxvyvz(
                v_los_kms,
                pml_masyr,pmb_masyr,
                l_deg,b_deg,
                d_kpc,
                XYZ=False,degree=True
                )
U_HC_kms = vxvyvz[:,0]
V_HC_kms = vxvyvz[:,1]
W_HC_kms = vxvyvz[:,2]
# use this to go to cylindrical coords used in Trick+19
#______(X,Y,Z) --> Galactic, Galactocentric cylindrical coordinates (R,phi,z)______:
Rzphi= coords.XYZ_to_galcencyl(
                X_HC_kpc, Y_HC_kpc, Z_HC_kpc, 
                Xsun=X_GC_sun_kpc,Zsun=Z_GC_sun_kpc
                )
R_kpc   = Rzphi[:,0]
phi_rad = Rzphi[:,1]
z_kpc   = Rzphi[:,2]

#______(vx,vy,vz) & (x,y,z) --> (vR,vT,vz)______
vRvTvZ = coords.vxvyvz_to_galcencyl(
                U_HC_kms, 
                V_HC_kms, 
                W_HC_kms, 
                R_kpc,
                phi_rad, 
                z_kpc,
                Xsun=X_GC_sun_kpc,Zsun=Z_GC_sun_kpc,
                vsun=[vX_GC_sun_kms,vY_GC_sun_kms,vZ_GC_sun_kms], 
                galcen=True
                )
vR_kms = vRvTvZ[:,0]
vT_kms = vRvTvZ[:,1]
vz_kms = vRvTvZ[:,2]

R   = R_kpc    / _REFR0   #Galactocentric radius
vR  = vR_kms   / _REFV0   #radial velocity
phi = phi_rad             #Galactocentric azimuth angle (not needed for actions in axisymmetric potential)
vT  = vT_kms / _REFV0     #tangential velocity
z   = z_kpc   / _REFR0    #height above plane
vz  = vz_kms  / _REFV0    #vertical velocity
aAS = actionAngleStaeckel(
        pot   = pot,        #potential
        delta = 0.45,       #focal length of confocal coordinate system
        c     = True        #use C code (for speed)
        )
jR,lz,jz = aAS(R,vR,vT,z,vz)

# print("Radial   action  J_R = ", jR*_REFR0*_REFV0,"\t kpc km/s")
# print("Vertical action  J_z = ", jz*_REFR0*_REFV0,"\t kpc km/s")
# print("Angular momentum L_z = ", lz*_REFR0*_REFV0,"\t kpc km/s")